### CybORG Action Space

Without the use of wrappers,  CybORG actions need to be constructed by the agent before being passed in. If you are not interested this we suggest you skip to the wrapper tutorial.

The action space is updated every step and can be found as a dictionary in the results object. Because this dictionary is quite large, we will only print the keys below.

In [1]:
import inspect
from pprint import pprint
from CybORG import CybORG

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario_reduced_observation_2112_manual.yaml'


env = CybORG(path, 'sim')

results = env.reset(agent='Red')
action_space = results.action_space
pprint(action_space.keys())

c:\Users\c21122256\PycharmProjects\CAGE-Challenge-1\venv\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


dict_keys(['action', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname'])


The CybORG action space is divided into "actions" and "parameters". Actions represent the use of specific cyber tools (for example a network scanning tool like nmap), while parameters represent the inputs the tool requires to function (to scan the interfaces of a host with nmap, you need to provide the ip address of the host).

The "actions" are located under the 'action' key in the action_space dictionary.

In [2]:
actions = action_space['action']
pprint(actions)

{<class 'CybORG.Shared.Actions.Action.Sleep'>: True,
 <class 'CybORG.Shared.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>: True,
 <class 'CybORG.Shared.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>: True,
 <class 'CybORG.Shared.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>: True,
 <class 'CybORG.Shared.Actions.AbstractActions.Impact.Impact'>: True,
 <class 'CybORG.Shared.Actions.AbstractActions.PrivilegeEscalate.PrivilegeEscalate'>: True}


We can see that our actions are each custom classes that form the keys of the above dictionary. The values specify whether this action is currently valid. In Scenario 1b, this value will always be True.

The remaining keys in the scenario dictionary represent different classes of parameters. For example, if we examine the 'ip_address' key we will get a dictionary whose keys are the various ip_addresses on the network. The values are again booleans, which represents whether Red knows about this ip_address or not.

In [3]:
ips = action_space['ip_address']
pprint(ips)

{IPv4Address('10.0.38.68'): False,
 IPv4Address('10.0.38.74'): False,
 IPv4Address('10.0.120.56'): False,
 IPv4Address('10.0.120.60'): True,
 IPv4Address('10.0.230.131'): False,
 IPv4Address('10.0.230.132'): False,
 IPv4Address('10.0.230.140'): False}


To construct an action, we choose (or import) an action class, then instantiate it by passing in the necessary parameters. These can be found HERE.

In [4]:
import random
from CybORG.Shared.Actions import DiscoverNetworkServices
unknown_ips = [ip for ip in ips if not ips[ip]]
ip = random.choice(unknown_ips)

action = DiscoverNetworkServices(session=0,agent='Red',ip_address=ip)

We have deliberately chosen to scan an ip address that Red Agent doesn't know about. Although randomly guessing an ip address to scan is possible in the real world, we have decided it is out of scope for Scenario1b and so this action will always fail. If you want to expose your agent to the action space, you should filter out all parameters with False values first.

In [5]:
results = env.step(action=action,agent='Red')
print(results.observation)

{'success': <TrinaryEnum.FALSE: 3>}


### Red Actions

We will now take a detailed look at Red Team's actions and understand what they do. Red's actions are listed below.

In [6]:
pprint([action.__name__ for action in actions if actions[action]])

['Sleep',
 'DiscoverRemoteSystems',
 'DiscoverNetworkServices',
 'ExploitRemoteService',
 'PrivilegeEscalate',
 'Impact']


The Sleep action does nothing and requires no parameters.

In [7]:
from CybORG.Shared.Actions import *

action = Sleep()
results = env.step(action=action,agent='Red')
print(results.observation)

{'success': <TrinaryEnum.UNKNOWN: 2>}


The DiscoverRemoteSystems action represents a ping sweep and takes in a subnet parameter to return all ips active on that subnet. Note how we pull the 

In [8]:
subnets = action_space['subnet']
known_subnets = [subnet for subnet in subnets if subnets[subnet]]
subnet = known_subnets[0]

action = DiscoverRemoteSystems(subnet = subnet, session=0,agent='Red')
results = env.step(action=action,agent='Red')
pprint(results.observation)

{'10.0.120.56': {'Interface': [{'IP Address': IPv4Address('10.0.120.56'),
                                'Subnet': IPv4Network('10.0.120.48/28')}]},
 '10.0.120.60': {'Interface': [{'IP Address': IPv4Address('10.0.120.60'),
                                'Subnet': IPv4Network('10.0.120.48/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}


The DiscoverNetworkServices action represents a port scan and takes in an ip address parameter to return a list of open ports and their respective services. These will be represented in the observation as new connections. The Red team must have discovered the ip address using the DiscoverRemoteSystems action in order for this action to succeed.

In [9]:
known_ips = [ip for ip in ips if ips[ip]]
ip = random.choice(known_ips)
action = DiscoverNetworkServices(ip_address=ip,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

{'10.0.120.56': {'Interface': [{'IP Address': IPv4Address('10.0.120.56')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.120.56'),
                                                 'local_port': 22}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.120.56'),
                                                 'local_port': 135}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.120.56'),
                                                 'local_port': 3389}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.120.56'),
                                                 'local_port': 445}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.120.56'),
                                                 'local_port': 139}]}]},
 'success': <TrinaryEnum.TRUE: 1>}


The ExploitRemoteService represents the use of a service exploit to obtain a reverse shell on the host. It requires an ip address as an input parameter and creates a new shell on the target host. 

CybORG actually models several different types of real-world exploits and this action chooses between them depending on the services available and the operating system of the host. This action will only ever succeed if the host's ip address has been discovered by Red team.

Usually the shell created by this action will be a shell with user privileges, but some exploits, such as EternalBlue, give SYSTEM access to a windows machine. In this case, performing the Privilege Escalation action afterwards is unnecessary, although our rules-based agents always will.

In [10]:
action = ExploitRemoteService(ip_address=ip,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

{'10.0.120.56': {'Interface': [{'IP Address': IPv4Address('10.0.120.56')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.120.56'),
                                                 'local_port': 52075,
                                                 'remote_address': IPv4Address('10.0.120.60'),
                                                 'remote_port': 4444}],
                                'Process Type': <ProcessType.REVERSE_SESSION: 9>},
                               {'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                 'local_address': IPv4Address('10.0.120.56'),
                                                 'local_port': 139}],
                                'Process Type': <ProcessType.SMB: 4>}],
                 'Sessions': [{'Agent': 'Red',
                               'ID': 1,
                               'Type': <SessionType.RED_REVERSE_SHELL: 11>}],
                 'System i

The PrivilegeEscalate represents the use of malware to establish a privileged shell with root (Linux) or SYSTEM (Windows) privileges. This action requires a user shell to be on the target host.

This action has the potential to reveals information about hosts on other subnets, which can then be scanned and exploited.

In [11]:
hostname = results.observation[str(ip)]['System info']['Hostname']
action = PrivilegeEscalate(hostname=hostname,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

{'Enterprise1': {'Interface': [{'IP Address': IPv4Address('10.0.230.132')}]},
 'User1': {'Interface': [{'IP Address': IPv4Address('10.0.120.56'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.120.48/28')}],
           'Processes': [{'PID': 9804, 'Username': 'SYSTEM'}],
           'Sessions': [{'Agent': 'Red',
                         'ID': 1,
                         'PID': 9804,
                         'Timeout': 0,
                         'Type': <SessionType.RED_REVERSE_SHELL: 11>,
                         'Username': 'SYSTEM'}]},
 'success': <TrinaryEnum.TRUE: 1>}


The Impact action represents the degredation of services. It requires a hostname input parameter, but will only work on the 'OpServer0' host on the Operational subnet and needs to be continually run in order to have an ongoing effect.

In [12]:
from CybORG.Agents.SimpleAgents.B_line_2112 import B_lineAgent as B_lineAgent_2112

results = env.reset(agent='Red')
obs = results.observation
action_space = results.action_space
agent = B_lineAgent_2112()

while True:
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    if action.__class__.__name__ == 'Impact':
        print(action)
        print(obs)
        break

KeyboardInterrupt: 

## Blue Actions

We will now take a look at Blue Team's actions and how they interact with those of Red Team.

In [ ]:
env = CybORG(path, 'sim',agents={'Red':B_lineAgent})
results = env.reset('Blue')
actions = results.action_space['action']

pprint([action.__name__ for action in actions if actions[action]])

['Sleep', 'Monitor', 'Analyse', 'Remove', 'Restore']


Similar to Red Team, the sleep action for Blue Team has no effect. However, like all Blue Team actions it does have passive monitoring capabilities as explained in the observation tutorial.

In [ ]:
action = Sleep()

for i in range(4):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    if i == 2:
        # The particular obs we want
        pprint(obs)

{'User1': {'Interface': [{'IP Address': IPv4Address('10.0.127.195')}],
           'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.127.195'),
                                           'local_port': 139,
                                           'remote_address': IPv4Address('10.0.127.201'),
                                           'remote_port': 59486}]},
                         {'Connections': [{'local_address': IPv4Address('10.0.127.195'),
                                           'local_port': 57615,
                                           'remote_address': IPv4Address('10.0.127.201'),
                                           'remote_port': 4444}],
                          'PID': 29546}],
           'System info': {'Architecture': <Architecture.x64: 2>,
                           'Hostname': 'User1',
                           'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                           'OSType': <OperatingSystemType.W

As explained by the Observation Tutorial, the Analyse action can detect malware files on a single host. This mimics the use of a malware-detection tool such as DensityScout. Like all of Blue's actions, it requires a hostname parameter.

We can see below that the action discovers malware on 'User1' as well as the passive monitoring picking up an exploit used Enterprise 1.

In [ ]:
action = Analyse(hostname='User1',session=0,agent='Blue')

for i in range(2):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    if i == 1:
        pprint(obs)

{'Enterprise1': {'Interface': [{'IP Address': IPv4Address('10.0.7.125')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.7.125'),
                                                 'local_port': 443,
                                                 'remote_address': IPv4Address('10.0.127.201'),
                                                 'remote_port': 51211}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.7.125'),
                                                 'local_port': 52627,
                                                 'remote_address': IPv4Address('10.0.127.201'),
                                                 'remote_port': 4444}],
                                'PID': 3109}],
                 'System info': {'Architecture': <Architecture.x64: 2>,
                                 'Hostname': 'Enterprise1',
                                 'OSDistribution': <OperatingSystemDistribution.WIND

The Remove action allows Blue Team to remove any of Red's user-level shells, simulating the act of killing it as a process. It will not remove a privileged shell. This is because privileged shells in Scenario1b are assumed to be persistent, meaning that if you remove them they will immediately come back.

We can see below that the Red agent attempts to PrivilegeEscalate, but this fails as its shell has been killed. The next turn it has to re-exploit the machine. Notice the use of the get_last_action method to work out what Red's last move was.

In [ ]:
action = Remove(hostname='Enterprise1', session=0, agent='Blue')

for i in range(2):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    pprint(obs)
    print(73*'-')
    print(env.get_last_action('Red'))
    print(73*'*')

{'success': <TrinaryEnum.TRUE: 1>}
-------------------------------------------------------------------------
PrivilegeEscalate Enterprise1
*************************************************************************
{'Enterprise1': {'Interface': [{'IP Address': IPv4Address('10.0.7.125')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.7.125'),
                                                 'local_port': 443,
                                                 'remote_address': IPv4Address('10.0.127.201'),
                                                 'remote_port': 54032}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.7.125'),
                                                 'local_port': 51744,
                                                 'remote_address': IPv4Address('10.0.127.201'),
                                                 'remote_port': 4444}],
                                'PID': 27274}],
  

The Restore action represents reverting the system to a known baseline. This will restore a host to the state it was at the beginning of the game. This will wipe all of Red's shells away, with the notable exception of Red's starting host 'User0', which has been baselined into the system. Although Restore is more powerful than Remove, it necessarily causes some disruption on the network so has a large negative penalty associated by using it.

Below we can see that the Analyse action detects malware on 'User1', but this disappears after restore has been used.

In [ ]:
for i in range(10):
    env.step() # So Red's actions don't interfere

action = Analyse(hostname='User1', session=0, agent='Blue')
results = env.step(action=action,agent='Blue')
obs = results.observation
pprint(obs)
    
action = Restore(hostname='User1', session=0, agent='Blue')
results = env.step(action=action,agent='Blue')
obs = results.observation
pprint(obs)

action = Analyse(hostname='User1', session=0, agent='Blue')
obs = results.observation
pprint(obs)

{'User1': {'Files': [{'Density': 0.9,
                      'File Name': 'cmd.exe',
                      'Known File': <FileType.UNKNOWN: 1>,
                      'Known Path': <Path.TEMP: 5>,
                      'Path': 'C:\\temp\\',
                      'Signed': False}]},
 'success': <TrinaryEnum.TRUE: 1>}
{'success': <TrinaryEnum.TRUE: 1>}
{'success': <TrinaryEnum.TRUE: 1>}


### Miscellany

If you create an action that doesn't make any sense within the current scenario, CybORG will accept it, but automatically convert it to an Invalid Action. These actions automatically give a reward of -0.1.

In [ ]:
action = Analyse(hostname = "Uncle Ted's Macbook", session = 1.1, agent='Cyan')

results = env.step(action=action,agent='Blue')

print(results.action)
print(results.reward)

InvalidAction
-13.1
